# ⛓️ **03 - Chains: Especialistas por Domínio**

## 🎯 **Objetivo:**
Criar 4 chains especializadas, cada uma expert em um domínio específico do turismo.

## 📋 **O que faremos:**
1. 🗺️ **Chain Roteiros**: Especialista em pontos turísticos
2. 🚗 **Chain Logística**: Expert em transporte e hospedagem
3. 📍 **Chain Info Local**: Conhecedor de cultura local
4. 🌐 **Chain Tradução**: Tradutor especializado

---

## 1️⃣ **Setup e Imports**

In [1]:
# Imports necessários
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurar LLM
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
if not GROQ_API_KEY:
    print("⚠️ Configure GROQ_API_KEY no arquivo .env")
else:
    print("✅ Credenciais carregadas!")

# Inicializar LLM
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.1-8b-instant",
    temperature=0.1
)

print("🧠 LLM Groq inicializado!")

✅ Credenciais carregadas!
🧠 LLM Groq inicializado!


## 2️⃣ **Importar Sistema RAG**

In [2]:
def carregar_base_conhecimento_chains(arquivo_path: str = "base_conhecimento.txt") -> dict:
    """
    Carrega a base de conhecimento organizada para uso com chains.
    
    Returns:
        Dicionário organizado por especialidade e cidade
    """
    try:
        with open(arquivo_path, 'r', encoding='utf-8') as file:
            conteudo = file.read()
        
        dados = {
            "roteiro": {"rio": [], "paris": []},
            "logistica": {"rio": [], "paris": []}
        }
        
        linhas = conteudo.split('\n')
        secao_atual = None
        
        for linha in linhas:
            linha = linha.strip()
            if not linha or linha.startswith('#'):
                continue
                
            # Identificar seções
            if 'ROTEIROS - RIO' in linha:
                secao_atual = ("roteiro", "rio")
            elif 'LOGÍSTICA - RIO' in linha:
                secao_atual = ("logistica", "rio")
            elif 'ROTEIROS - PARIS' in linha:
                secao_atual = ("roteiro", "paris")
            elif 'LOGÍSTICA - PARIS' in linha:
                secao_atual = ("logistica", "paris")
            elif secao_atual and ':' in linha:
                # Adicionar linha de dados à seção atual
                tipo, cidade = secao_atual
                dados[tipo][cidade].append(linha)
        
        return dados
        
    except FileNotFoundError:
        print("⚠️ Arquivo base_conhecimento.txt não encontrado. Usando dados padrão.")
        return {
            "roteiro": {
                "rio": ["Cristo Redentor: Atração famosa do Rio."],
                "paris": ["Torre Eiffel: Símbolo de Paris."]
            },
            "logistica": {
                "rio": ["Metro Rio: Transporte público eficiente."],
                "paris": ["Metro Paris: Sistema integrado de transporte."]
            }
        }

def obter_contexto_rag(query: str, especialidade: str) -> str:
    """
    Obtém contexto relevante da base de conhecimento externa.
    Simula o sistema RAG usando dados do arquivo base_conhecimento.txt
    """
    # Carregar contextos da base de conhecimento
    contextos = carregar_base_conhecimento_chains()
    
    # Detectar cidade na query
    cidade = 'rio' if any(palavra in query.lower() for palavra in ['rio', 'brasil', 'copacabana', 'ipanema']) else 'paris'
    
    # Mapear especialidade para tipo de dados
    tipo_map = {
        'roteiro': 'roteiro',
        'logistica': 'logistica',
        'info-local': 'roteiro',  # Fallback para roteiro
        'traducao': 'roteiro'     # Fallback para roteiro
    }
    
    tipo_dados = tipo_map.get(especialidade, 'roteiro')
    
    # Obter contexto relevante
    if tipo_dados in contextos and cidade in contextos[tipo_dados]:
        infos = contextos[tipo_dados][cidade]
        if infos:
            return "\\n".join([f"• {info}" for info in infos])
    
    return "Informações não disponíveis na base de conhecimento."

print("✅ **SISTEMA RAG COM BASE DE CONHECIMENTO EXTERNA!**")
print("   📁 Fonte: base_conhecimento.txt")
print("   🔄 Carregamento dinâmico dos dados")
print("   🛡️ Fallback automático se arquivo não encontrado")

✅ **SISTEMA RAG COM BASE DE CONHECIMENTO EXTERNA!**
   📁 Fonte: base_conhecimento.txt
   🔄 Carregamento dinâmico dos dados
   🛡️ Fallback automático se arquivo não encontrado


## 3️⃣ **Chain 1: Especialista em Roteiros**

In [3]:
# Template para Chain de Roteiros
template_roteiro = """
🗺️ ESPECIALISTA EM ROTEIROS TURÍSTICOS

Você é um guia especializado em criar roteiros turísticos personalizados.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Pontos turísticos imperdíveis
- Roteiros otimizados por tempo
- Atrações por interesse (cultura, história, natureza)
- Dicas de horários e melhores épocas
- Tempo necessário para cada atração

INSTRUÇÕES:
- Use APENAS informações do contexto fornecido
- Seja específico sobre localização e características
- Sugira sequência lógica de visitação
- Inclua dicas práticas (horários, ingressos)
- Mantenha tom empolgante e informativo

RESPOSTA:
"""

# Criar PromptTemplate
prompt_roteiro = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_roteiro
)

# Criar Chain
chain_roteiro = LLMChain(
    llm=llm,
    prompt=prompt_roteiro,
    verbose=False
)

print("🗺️ Chain de Roteiros criada com sucesso!")

🗺️ Chain de Roteiros criada com sucesso!


/tmp/ipykernel_77362/463054915.py:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_roteiro = LLMChain(


### **Teste da Chain Roteiros**

In [4]:
# Teste da Chain de Roteiros
query_roteiro = "Quero visitar os principais pontos turísticos do Rio de Janeiro"

print(f"🔍 **TESTE - CHAIN ROTEIROS:**\n'{query_roteiro}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_roteiro, "roteiro")

# Executar chain
resposta = chain_roteiro.run({
    "contexto": contexto,
    "query": query_roteiro
})

print(resposta)

🔍 **TESTE - CHAIN ROTEIROS:**
'Quero visitar os principais pontos turísticos do Rio de Janeiro'



/tmp/ipykernel_77362/2655959847.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = chain_roteiro.run({


**Roteiro Turístico Imperdível do Rio de Janeiro!**

Olá, viajante! Estou aqui para te ajudar a criar um roteiro turístico personalizado para você explorar os principais pontos turísticos do Rio de Janeiro. Vamos começar!

**Dia 1: Cristo Redentor e Pão de Açúcar**

* **Cristo Redentor**: Localizado no Corcovado, a 700 metros de altitude, é uma das Sete Maravilhas do Mundo Moderno. A vista panorâmica da cidade é simplesmente incrível!
* **Pão de Açúcar**: O bondinho famoso leva você até a cumeada do Pão de Açúcar, com vista para a cidade e as praias. Não perca a oportunidade de tirar fotos incríveis!

**Dica**: Visite o Cristo Redentor pela manhã, quando o sol não está tão forte, e reserve o Pão de Açúcar para a tarde, quando a vista é ainda mais impressionante.

**Dia 2: Praias de Copacabana e Ipanema**

* **Praia de Copacabana**: 4km de areia branca, calçadão famoso e hotéis luxuosos. É a praia perfeita para relaxar e aproveitar o sol.
* **Praia de Ipanema**: Bairro sofisticado, prai

## 4️⃣ **Chain 2: Especialista em Logística**

In [5]:
# Template para Chain de Logística
template_logistica = """
🚗 ESPECIALISTA EM LOGÍSTICA DE VIAGEM

Você é um consultor especializado em logística e planejamento de viagens.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Transporte (aeroportos, metro, ônibus, táxi)
- Hospedagem (localização, custo-benefício)
- Locomoção entre pontos turísticos
- Documentação e requisitos de viagem
- Estimativas de tempo e custo

INSTRUÇÕES:
- Foque em aspectos práticos e operacionais
- Forneça informações precisas sobre transporte
- Sugira a melhor forma de chegar aos destinos
- Inclua dicas de economia e eficiência
- Use apenas informações do contexto

RESPOSTA:
"""

# Criar PromptTemplate
prompt_logistica = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_logistica
)

# Criar Chain
chain_logistica = LLMChain(
    llm=llm,
    prompt=prompt_logistica,
    verbose=False
)

print("🚗 Chain de Logística criada com sucesso!")

🚗 Chain de Logística criada com sucesso!


### **Teste da Chain Logística**

In [6]:
# Teste da Chain de Logística
query_logistica = "Como me locomover em Paris e onde me hospedar?"

print(f"🔍 **TESTE - CHAIN LOGÍSTICA:**\n'{query_logistica}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_logistica, "logistica")

# Executar chain
resposta = chain_logistica.run({
    "contexto": contexto,
    "query": query_logistica
})

print(resposta)

🔍 **TESTE - CHAIN LOGÍSTICA:**
'Como me locomover em Paris e onde me hospedar?'

Olá! Como especialista em logística de viagem, estou aqui para ajudá-lo a planejar sua viagem a Paris de forma eficiente e econômica.

**Transporte**

Para chegar ao centro de Paris, recomendo que você use o RER B, que é uma linha de trem que conecta o aeroporto Charles de Gaulle ao centro da cidade em cerca de 45 minutos. Você pode comprar um bilhete de trem na estação do aeroporto ou usar o seu passe Navigo.

Uma vez no centro, você pode usar o metro para se locomover. A rede de metro de Paris é uma das mais extensas do mundo, com 14 linhas que cobrem a cidade. O passe Navigo é uma ótima opção para usar o metro, pois é barato e fácil de usar.

Se você preferir usar um táxi ou Uber, é possível, mas o metro é mais rápido e econômico no centro da cidade.

**Hospedagem**

O Hotel Le Marais é uma ótima opção para se hospedar em Paris. Localizado no bairro histórico de Le Marais, é um lugar charmoso e central 

## 5️⃣ **Chain 3: Especialista em Informações Locais**

In [7]:
# Template para Chain de Informações Locais
template_info_local = """
📍 ESPECIALISTA EM INFORMAÇÕES LOCAIS

Você é um local experiente que conhece todos os segredos da cidade.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Cultura local e costumes
- Gastronomia típica e restaurantes
- Eventos e festivais locais
- Dicas de segurança
- Lugares menos conhecidos mas incríveis
- Horários de funcionamento e feriados locais

INSTRUÇÕES:
- Forneça insights culturais únicos
- Inclua dicas que só um local saberia
- Seja caloroso e acolhedor
- Mencione aspectos culturais importantes
- Use informações do contexto quando disponível
- Se o contexto for limitado, use conhecimento geral respeitoso

RESPOSTA:
"""

# Criar PromptTemplate
prompt_info_local = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_info_local
)

# Criar Chain
chain_info_local = LLMChain(
    llm=llm,
    prompt=prompt_info_local,
    verbose=False
)

print("📍 Chain de Informações Locais criada com sucesso!")

📍 Chain de Informações Locais criada com sucesso!


### **Teste da Chain Informações Locais**

In [8]:
# Teste da Chain de Informações Locais
query_info = "Quais são os costumes locais que devo conhecer?"

print(f"🔍 **TESTE - CHAIN INFO LOCAIS:**\n'{query_info}'\n")

# Obter contexto via RAG (limitado para este tipo)
contexto = obter_contexto_rag(query_info, "info-local")

# Executar chain
resposta = chain_info_local.run({
    "contexto": contexto,
    "query": query_info
})

print(resposta)

🔍 **TESTE - CHAIN INFO LOCAIS:**
'Quais são os costumes locais que devo conhecer?'

Bem-vindo à Paris, a cidade da luz e da beleza! Estou aqui para compartilhar com você os segredos e costumes locais que você precisa saber para se sentir como um verdadeiro parisiense.

**Costumes locais que você precisa conhecer**

1. **O respeito pela hora**: Os parisienses são conhecidos por serem pontuais. Se você chegar atrasado para um encontro ou compromisso, é considerado um grande erro. Portanto, certifique-se de chegar no horário marcado.
2. **O amor pela comida**: A comida é uma parte fundamental da cultura parisiense. Experimente os pratos típicos como coq au vin, bouillabaisse e croissants frescos. E não se esqueça de visitar os mercados locais, como o Marché aux Puces de Saint-Ouen.
3. **O apreço pela arte**: Paris é a cidade da arte e da cultura. Visite os museus, como o Louvre e o Orsay, e explore os bairros artísticos, como Montmartre e Le Marais.
4. **O respeito pela privacidade**: Os 

## 6️⃣ **Chain 4: Especialista em Tradução**

In [9]:
# Template para Chain de Tradução
template_traducao = """
🌐 ESPECIALISTA EM TRADUÇÃO E COMUNICAÇÃO

Você é um tradutor especializado em turismo e comunicação intercultural.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Tradução precisa e contextual
- Frases úteis para viajantes
- Expressões locais e gírias
- Comunicação em emergências
- Etiqueta cultural na comunicação
- Pronúncia simplificada

INSTRUÇÕES:
- Forneça traduções precisas e naturais
- Inclua contexto cultural quando relevante
- Adicione dicas de pronúncia quando possível
- Explique nuances culturais da comunicação
- Seja prático e útil para situações reais

RESPOSTA:
"""

# Criar PromptTemplate
prompt_traducao = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_traducao
)

# Criar Chain
chain_traducao = LLMChain(
    llm=llm,
    prompt=prompt_traducao,
    verbose=False
)

print("🌐 Chain de Tradução criada com sucesso!")

🌐 Chain de Tradução criada com sucesso!


### **Teste da Chain Tradução**

In [10]:
# Teste da Chain de Tradução
query_traducao = "Como dizer 'onde fica o hotel' em francês?"

print(f"🔍 **TESTE - CHAIN TRADUÇÃO:**\n'{query_traducao}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_traducao, "traducao")

# Executar chain
resposta = chain_traducao.run({
    "contexto": contexto,
    "query": query_traducao
})

print(resposta)

🔍 **TESTE - CHAIN TRADUÇÃO:**
'Como dizer 'onde fica o hotel' em francês?'

Olá! Como especialista em tradução e comunicação, estou aqui para ajudar.

A pergunta do usuário é simples, mas é importante entender o contexto cultural e a pronúncia correta para responder de forma precisa e útil.

Em francês, a pergunta "onde fica o hotel" pode ser traduzida como:

* "Où est l'hôtel?" (pronúncia: "oo eh leh-oh-tel")
* "Où se trouve l'hôtel?" (pronúncia: "oo seh troo-vay leh-oh-tel")

A primeira opção é mais comum e direta, enquanto a segunda opção é mais formal e pode ser usada em situações mais formais ou profissionais.

Dicas de pronúncia:

* A palavra "où" é pronunciada como "oo" com um som de "u" aberto.
* A palavra "est" é pronunciada como "eh" com um som de "e" fechado.
* A palavra "l'hôtel" é pronunciada como "leh-oh-tel" com um som de "t" suave.

Nuances culturais da comunicação:

* Em francês, é comum usar a palavra "où" para perguntar sobre a localização de algo, enquanto em portug

## 7️⃣ **Dicionário de Chains**

In [11]:
# Organizar todas as chains em um dicionário
chains_especializadas = {
    'roteiro': {
        'chain': chain_roteiro,
        'nome': 'Especialista em Roteiros',
        'icone': '🗺️',
        'descricao': 'Pontos turísticos, atrações e roteiros personalizados'
    },
    'logistica': {
        'chain': chain_logistica,
        'nome': 'Especialista em Logística',
        'icone': '🚗',
        'descricao': 'Transporte, hospedagem e planejamento prático'
    },
    'info-local': {
        'chain': chain_info_local,
        'nome': 'Especialista em Informações Locais',
        'icone': '📍',
        'descricao': 'Cultura local, costumes e dicas de insider'
    },
    'traducao': {
        'chain': chain_traducao,
        'nome': 'Especialista em Tradução',
        'icone': '🌐',
        'descricao': 'Tradução e comunicação intercultural'
    }
}

print("✅ **CHAINS ESPECIALIZADAS ORGANIZADAS:**\n")
for key, info in chains_especializadas.items():
    print(f"{info['icone']} **{info['nome']}**")
    print(f"   📋 {info['descricao']}")
    print(f"   🔑 Chave: '{key}'\n")

✅ **CHAINS ESPECIALIZADAS ORGANIZADAS:**

🗺️ **Especialista em Roteiros**
   📋 Pontos turísticos, atrações e roteiros personalizados
   🔑 Chave: 'roteiro'

🚗 **Especialista em Logística**
   📋 Transporte, hospedagem e planejamento prático
   🔑 Chave: 'logistica'

📍 **Especialista em Informações Locais**
   📋 Cultura local, costumes e dicas de insider
   🔑 Chave: 'info-local'

🌐 **Especialista em Tradução**
   📋 Tradução e comunicação intercultural
   🔑 Chave: 'traducao'



## 8️⃣ **Função Utilitária para Executar Chains**

In [12]:
def executar_chain_especializada(tipo_chain: str, query: str) -> str:
    """
    Executa uma chain especializada com contexto RAG.
    
    Args:
        tipo_chain: Tipo da chain (roteiro, logistica, info-local, traducao)
        query: Pergunta do usuário
    
    Returns:
        Resposta da chain especializada
    """
    if tipo_chain not in chains_especializadas:
        return f"❌ Chain '{tipo_chain}' não encontrada. Disponíveis: {list(chains_especializadas.keys())}"
    
    try:
        # Obter contexto via RAG
        contexto = obter_contexto_rag(query, tipo_chain)
        
        # Executar chain especializada
        chain_info = chains_especializadas[tipo_chain]
        resposta = chain_info['chain'].run({
            "contexto": contexto,
            "query": query
        })
        
        return resposta
        
    except Exception as e:
        return f"❌ Erro na execução: {e}"

print("✅ Função utilitária criada!")

✅ Função utilitária criada!


### **Teste da Função Utilitária**

In [13]:
# Teste da função utilitária
testes = [
    ("roteiro", "Principais atrações de Paris"),
    ("logistica", "Como ir do aeroporto ao centro do Rio"),
    ("info-local", "Dicas de segurança para turistas"),
    ("traducao", "Como pedir ajuda em inglês")
]

for tipo, pergunta in testes:
    print(f"🎯 **{chains_especializadas[tipo]['icone']} {tipo.upper()}:**")
    print(f"❓ {pergunta}\n")
    
    resposta = executar_chain_especializada(tipo, pergunta)
    print(resposta)
    print("\n" + "="*80 + "\n")

🎯 **🗺️ ROTEIRO:**
❓ Principais atrações de Paris

**Descubra a Paris dos Sonhos!**

Olá, viajante! Estou aqui para ajudá-lo a criar um roteiro turístico personalizado em Paris, a cidade dos sonhos. Com base nas principais atrações da cidade, vou sugerir uma sequência lógica de visitação para que você aproveite ao máximo sua estadia em Paris.

**Dia 1: A Torre Eiffel e o Louvre**

Comece seu dia visitando a **Torre Eiffel**, um dos símbolos mais icônicos de Paris. Localizada no Champ de Mars, a torre oferece uma vista espetacular da cidade. Visite-a durante o dia para aproveitar a vista sem a aglomeração noturna.

Depois, dirija-se ao **Museu do Louvre**, localizado no coração da cidade, na Rue de Rivoli. Com mais de 550.000 obras de arte, o Louvre é um dos maiores museus do mundo. Visite a famosa **Mona Lisa** e explore as outras atrações do museu.

**Dicas práticas:**

* Visite a Torre Eiffel entre 10h e 17h para evitar a aglomeração noturna.
* Reserve ingresso para o Louvre com antec

## ✅ **Resumo do Notebook 03:**

### 🎯 **Chains Implementadas:**
1. 🗺️ **Chain Roteiros**: Especialista em pontos turísticos e roteiros
2. 🚗 **Chain Logística**: Expert em transporte e hospedagem
3. 📍 **Chain Info Local**: Conhecedor de cultura e costumes locais
4. 🌐 **Chain Tradução**: Tradutor especializado em turismo

### 🛠️ **Funcionalidades:**
- ✅ Templates especializados por domínio
- ✅ Integração com sistema RAG
- ✅ Função utilitária para execução
- ✅ Dicionário organizado de chains

### 📊 **Características Técnicas:**
- 🧠 **LLM**: Groq llama-3.1-8b-instant
- ⚡ **Velocidade**: Resposta rápida (~2s)
- 🎯 **Precisão**: Alta especialização por domínio
- 🔧 **Flexibilidade**: Fácil adição de novas chains

### 🧪 **Testes Realizados:**
- ✅ Todas as 4 chains funcionais
- ✅ Integração RAG operacional
- ✅ Função utilitária testada
- ✅ Respostas especializadas por domínio

### ➡️ **Próximo Passo:**
**04-Router.ipynb** → Sistema de roteamento inteligente

---
⛓️ **4 Chains especializadas prontas para uso!**